# Gotobi and RSI/MACD/MA Strategy Backtest Demo

Demonstrates running either the Gotobi FX settlement strategy or the RSI/MACD/MA strategy using NautilusTrader's BacktestEngine.
Loads USDJPY data from parquet files and runs a base strategy plus a comparison variant.


In [1]:
from pathlib import Path
import sys
import importlib

import pandas as pd
import plotly.graph_objects as go

from nautilus_trader.backtest.engine import BacktestEngineConfig
from nautilus_trader.common.config import LoggingConfig
from nautilus_trader.model.data import BarType


def find_project_root(start: Path) -> Path:
    for candidate in (start, *start.parents):
        if (candidate / "pyproject.toml").exists() and (candidate / "trader").exists():
            return candidate
    raise RuntimeError("Could not find project root containing pyproject.toml and trader/.")


PROJECT_ROOT = find_project_root(Path.cwd().resolve())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from trader import DataHandler, SIM

import trader.data.catalog as catalog
importlib.reload(catalog)
dataframe_to_nautilus_bars = catalog.dataframe_to_nautilus_bars

import trader.core.instruments as instruments
importlib.reload(instruments)
make_fx_pair = instruments.make_fx_pair

import trader.config.node as node
importlib.reload(node)
build_backtest_engine = node.build_backtest_engine

import trader.strategy.gotobi as gotobi
importlib.reload(gotobi)
GotobiStrategy = gotobi.GotobiStrategy
GotobiConfig = gotobi.GotobiConfig
GotobiWithSLStrategy = gotobi.GotobiWithSLStrategy
GotobiWithSLConfig = gotobi.GotobiWithSLConfig

import trader.strategy.rsi_macd_ma as rsi_macd_ma
importlib.reload(rsi_macd_ma)
RsiMacdMaStrategy = rsi_macd_ma.RsiMacdMaStrategy
RsiMacdMaConfig = rsi_macd_ma.RsiMacdMaConfig

print(f"Project root: {PROJECT_ROOT}")


Project root: S:\Github\Trading


## Load Data

In [2]:
# Load USDJPY parquet data

data_candidates = [
    PROJECT_ROOT / "data" / "USDJPY_1min_2024-01-01_2025-10-01.parquet",
    PROJECT_ROOT / "data" / "usdjpy_1min_2024-01-01_2025-10-01.parquet",
]
data_path = next((path for path in data_candidates if path.exists()), None)
if data_path is None:
    raise FileNotFoundError(f"Could not find USDJPY parquet in: {data_candidates}")

handler = DataHandler()
df = handler.load_parquet(str(data_path), tz="Asia/Tokyo")

print(f"Loaded {len(df):,} bars from {df.index[0]} to {df.index[-1]}")
print(f"Data path: {data_path}")


Loaded 644,043 bars from 2024-01-01 09:32:00+09:00 to 2025-10-01 08:59:00+09:00
Data path: S:\Github\Trading\data\USDJPY_1min_2024-01-01_2025-10-01.parquet


## Build Instruments and Bar Data

In [3]:
# Create USDJPY instrument
instrument = make_fx_pair("USDJPY", SIM, lot_size=100_000)

# Define bar type
bar_type = BarType.from_str(f"{instrument.id}-1-MINUTE-MID-EXTERNAL")

# Convert DataFrame to NautilusTrader bars
bars = dataframe_to_nautilus_bars(df, bar_type, price_precision=instrument.price_precision)
print(f"Converted {len(bars):,} bars for {bar_type}")


Converted 644,043 bars for USD/JPY.SIM-1-MINUTE-MID-EXTERNAL


## Run Base Strategy

Toggle `USE_RSI_MACD_MA` to switch between strategy families.


In [4]:
# Choose strategy family
USE_RSI_MACD_MA = False

if USE_RSI_MACD_MA:
    base_label = "RSI/MACD/MA"
    base_config = RsiMacdMaConfig(
        instrument_id=str(instrument.id),
        bar_type=str(bar_type),
        trade_size=50.0,
        rsi_period=14,
        rsi_oversold=30.0,
        rsi_overbought=70.0,
        macd_fast=12,
        macd_slow=26,
        macd_signal=9,
        ma_fast=20,
        ma_slow=50,
    )
    base_strategies = [RsiMacdMaStrategy(config=base_config)]
else:
    base_label = "Gotobi"
    long_config = GotobiConfig(
        instrument_id=str(instrument.id),
        bar_type=str(bar_type),
        entry_time="09:00:00",
        exit_time="09:55:00",
        trade_size=50.0,
        use_holidays=True,
        time_in_force="IOC"
    )

    short_config = GotobiConfig(
        instrument_id=str(instrument.id),
        bar_type=str(bar_type),
        entry_time="10:00:00",
        exit_time="10:45:00",
        trade_size=-50.0,
        use_holidays=True,
        time_in_force="IOC"
    )

    long_strat = GotobiStrategy(config=long_config)
    short_strat = GotobiStrategy(config=short_config)
    base_strategies = [long_strat, short_strat]

venue_currency = instrument.quote_currency.code
starting_balance = 150_000


def build_engine(strategies):
    return build_backtest_engine(
        instruments=[instrument],
        bars={bar_type: bars},
        strategies=strategies,
        venue=SIM,
        venue_currency=venue_currency,
        starting_balance=starting_balance,
        leverage=500.0,
        config=BacktestEngineConfig(
            logging=LoggingConfig(log_level="ERROR", log_colors=False),
        ),
    )


# Build and run base engine
engine = build_engine(base_strategies)
engine.run()
print(f"{base_label} backtest complete. Analysis currency: {venue_currency}")


Gotobi backtest complete. Analysis currency: JPY


## Run Comparison Strategy

When `USE_RSI_MACD_MA=True`, this runs an alternative RSI/MACD/MA parameter set.
When `USE_RSI_MACD_MA=False`, this runs Gotobi with stop loss.


In [5]:
# Configure and run comparison variant
if USE_RSI_MACD_MA:
    compare_label = "RSI/MACD/MA (Alt Params)"
    compare_config = RsiMacdMaConfig(
        instrument_id=str(instrument.id),
        bar_type=str(bar_type),
        trade_size=50.0,
        rsi_period=10,
        rsi_oversold=35.0,
        rsi_overbought=65.0,
        macd_fast=8,
        macd_slow=21,
        macd_signal=5,
        ma_fast=10,
        ma_slow=30,
    )
    compare_strategies = [RsiMacdMaStrategy(config=compare_config)]
else:
    compare_label = "Gotobi + Stop Loss"
    long_sl_config = GotobiWithSLConfig(
        instrument_id=str(instrument.id),
        bar_type=str(bar_type),
        entry_time="09:00:00",
        exit_time="09:55:00",
        trade_size=50.0,
        stop_loss_pct=0.001,
        use_holidays=True,
        time_in_force="IOC"
    )

    short_sl_config = GotobiWithSLConfig(
        instrument_id=str(instrument.id),
        bar_type=str(bar_type),
        entry_time="10:00:00",
        exit_time="10:45:00",
        trade_size=-50.0,
        stop_loss_pct=0.001,
        use_holidays=True,
        time_in_force="IOC"
    )

    long_sl = GotobiWithSLStrategy(config=long_sl_config)
    short_sl = GotobiWithSLStrategy(config=short_sl_config)
    compare_strategies = [long_sl, short_sl]

engine_compare = build_engine(compare_strategies)
engine_compare.run()
print(f"{compare_label} backtest complete.")


Gotobi + Stop Loss backtest complete.


In [6]:
analysis_currency = instrument.base_currency
analysis_currency_code = analysis_currency.code


def analyze_engine(engine, label):
    analyzer = engine.portfolio.analyzer
    pnl_stats = analyzer.get_performance_stats_pnls(currency=analysis_currency)
    returns_stats = analyzer.get_performance_stats_returns()
    general_stats = analyzer.get_performance_stats_general()

    # Build a resilient equity curve from returns; some analyzer outputs are unindexed/empty.
    returns = analyzer.returns()
    if returns is None:
        returns = pd.Series(dtype=float)
    else:
        returns = pd.Series(returns).dropna()

    if not returns.empty:
        returns = returns.sort_index()
        equity_curve = (1.0 + returns).cumprod() * starting_balance
    else:
        equity_curve = pd.Series([starting_balance], index=pd.Index([0], name="step"), dtype=float)

    equity_curve.name = label

    return pnl_stats, returns_stats, general_stats, equity_curve


stats_pnls_base, stats_returns_base, stats_general_base, equity_base = analyze_engine(engine, base_label)
stats_pnls_cmp, stats_returns_cmp, stats_general_cmp, equity_cmp = analyze_engine(engine_compare, compare_label)

stats_pnls = pd.DataFrame([stats_pnls_base, stats_pnls_cmp], index=[base_label, compare_label])
stats_returns = pd.DataFrame([stats_returns_base, stats_returns_cmp], index=[base_label, compare_label])
stats_general = pd.DataFrame([stats_general_base, stats_general_cmp], index=[base_label, compare_label])

equity_curves = pd.concat([equity_base, equity_cmp], axis=1).dropna(how="all")
if isinstance(equity_curves.index, pd.DatetimeIndex) and equity_curves.index.tz is not None:
    equity_curves.index = equity_curves.index.tz_convert("Asia/Tokyo")

fig = go.Figure()
for column in equity_curves.columns:
    fig.add_trace(
        go.Scatter(
            x=equity_curves.index,
            y=equity_curves[column],
            mode="lines",
            name=column,
        )
    )

fig.update_layout(
    title="Backtest Equity Curves",
    xaxis_title="Time",
    yaxis_title=f"Equity ({analysis_currency_code})",
    legend_title="Strategy",
    template="plotly_white",
)

fig
